In [1]:
import pandas as pd

# Definir las rutas
file_path = 'input/ranked_answers.csv'
output_csv = 'output/user_rankings_rated.csv'

# Cargar el archivo CSV
df = pd.read_csv(file_path)

# Crear la columna 'story_id' combinando 'title' y 'writer'
df['story_id'] = df['title'] + '_' + df['writer']

# Definir las columnas de puntuación a considerar
score_columns = ['atractivo', 'creatividad', 'gramaticalidad', 'relacion', 'informacion', 'sentido']

# Calcular la suma de puntuaciones para cada historia
df['total_score'] = df[score_columns].sum(axis=1)

def assign_ranking(group):
    group = group.copy()
    # Ordenar las historias por puntuación total de forma descendente
    group = group.sort_values('total_score', ascending=False)
    # Asignar ranking usando dense (mantener empates con la misma posición)
    group['ranking_pos'] = group['total_score'].rank(method='dense', ascending=False).astype(int)
    # El valor del ranking es la puntuación total
    group['ranking_value'] = group['total_score']
    # Calcular ranking_value normalizado por usuario (min-max)
    min_score = group['ranking_value'].min()
    max_score = group['ranking_value'].max()
    if max_score - min_score != 0:
        group['ranking_value_norm'] = (group['ranking_value'] - min_score) / (max_score - min_score)
    else:
        group['ranking_value_norm'] = 0.0
    return group

# Aplicar el ranking por usuario
df_ranked = df.groupby('user').apply(assign_ranking).reset_index(drop=True)

# Seleccionar las columnas deseadas
df_output = df_ranked[['user', 'story_id', 'ranking_pos', 'ranking_value', 'ranking_value_norm']]

# Guardar el resultado en un CSV
df_output.to_csv(output_csv, index=False)

print(f"CSV file created: {output_csv}")

CSV file created: output/user_rankings_rated.csv


/tmp/ipykernel_421045/3260149161.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_ranked = df.groupby('user').apply(assign_ranking).reset_index(drop=True)
